In [6]:
import merkletools
from web3 import Web3
from solcx import compile_files
import utils
import ram
import copy

In [7]:
contract_id, compiled_sol = compile_files(["test.sol"], output_values=['abi', 'bin']).popitem()

w3 = Web3(Web3.EthereumTesterProvider())

missmatch = w3.eth.contract(abi=compiled_sol['abi'], bytecode=compiled_sol['bin'])

tx_hash = missmatch.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

Missmatch = w3.eth.contract(address=tx_receipt.contractAddress, abi=compiled_sol['abi'])


In [8]:
code = ram.RamCoder(10)
code.load(5, 0)
code.load(7, 1)
code.load(1, 5)
code.add(0, 2, 2)
code.sub(1, 5, 1)
code.jnz(1, 2)
code.end()

ram1 = ram.Ram(code.source)
ram1()
ram1.rc_list

mt1 = utils.reduced_config_mt(ram1.rc_list)

corrupted_rc_list = copy.deepcopy(ram1.rc_list)
corrupted_rc_list[14]["args"] = [1, 2, 3]
corrupted_rc_list[15]["args"] = [1, 2, 3]
corrupted_rc_list[16]["args"] = [1, 2, 3]

mt2 = utils.reduced_config_mt(corrupted_rc_list)

mt2.get_leaf(11)

'62b40c8fabfee5a07bd5bc866a0c48ec74ea57d293ab40be0e156da9d07f4658'

In [11]:

rc = ram1.rc_list[12]
Missmatch.functions.send_committed_step(
    PC = rc['PC'], 
    OpCode = int(rc['OpCode']), 
    args = rc['args'], 
    args_values = utils.sterilize_ints(rc['args_values']), 
    args_proof = rc['args_proofs'],
    store_mt_root = rc['store_mt_root'],
    nextOpCode = int(ram1.rc_list[13]['OpCode']),
    # nextArgs = utils.sterilize_ints(ram1.rc_list[13]['args']),
    # next_op_proof = rc['next_op_proof']
    ).transact({'from': w3.eth.accounts[1]})

HexBytes('0x8c239c053a5d925df8afe0a14e136574beb66946661f5ca8ef57f16bd86dbeba')

In [10]:
import asyncio
mt = [mt1, mt2]

def handle_event(event, conractor):
    middle = event['args']['middle']
    Missmatch.functions.send_rc_proof(utils.translate_proof(mt[conractor-1], middle)).transact({'from': w3.eth.accounts[conractor]})
    print(f'conractor = {conractor} middle = {middle}')

def handle_verify_committed_step(event, rc_list, conractor):
    low = event['args']['low']
    rc = rc_list[low]
    Missmatch.functions.send_committed_step(
        PC = rc['PC'], 
        OpCode = int(rc['OpCode']), 
        args = rc['args'], 
        args_values = utils.sterilize_ints(rc['args_values']), 
        args_proof = rc['args_proofs'],
        store_mt_root = rc['store_mt_root'],
        nextOpCode = int(rc_list[low+1]['OpCode']),
        nextArgs = utils.sterilize_ints(rc_list[low+1]['args']),
        next_op_proof = rc['next_op_proof']
        ).transact({'from': w3.eth.accounts[conractor]})
    print(f'conractor = {conractor} low = {low}')
    

async def log_loop_c1(event_filter, event_filter_verify, poll_interval):
    for x in range(8):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 1)
        for event in event_filter_verify.get_new_entries():
            handle_verify_committed_step(event, ram1.rc_list, 1)
        await asyncio.sleep(poll_interval)

async def log_loop_c2(event_filter, event_filter_verify, poll_interval):
    for x in range(8):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 2)
        await asyncio.sleep(poll_interval)

async def main():
    event_filter_c1 = Missmatch.events.request_proof_of_middle.createFilter(fromBlock=1)
    event_filter_c2 = Missmatch.events.request_proof_of_middle.createFilter(fromBlock=1)
    event_filter_verift_c1 = Missmatch.events.request_proof_of_committed_step.createFilter(fromBlock=1)
    event_filter_verift_c2 = Missmatch.events.request_proof_of_committed_step.createFilter(fromBlock=1)

    await asyncio.gather(
        log_loop_c1(event_filter_c1, event_filter_verift_c1, 1),
        log_loop_c2(event_filter_c2, event_filter_verift_c2, 1))


await main()


ABIEventFunctionNotFound: ("The event 'request_proof_of_middle' was not found in this contract's abi. ", 'Are you sure you provided the correct contract abi?')

In [ ]:
event_filter = Missmatch.events.request_proof_of_committed_step.createFilter(fromBlock=1)
event_filter.get_new_entries()
type(ram1.rc_list[5]['args_values'][0])


numpy.int32